# Channel / Block / Input Sparsity Benchmark (FP16, forward+backward)

В этом ноутбуке сравниваются `nn.Conv2d` и базовая `TritonConv2d` (img2col→GEMM→col2img) при трёх режимах разрежения:
- Channel sparsity: обнуляем выходные каналы (Cout), уменьшается число столбцов в GEMM.
- Block sparsity: обнуляем фильтры блоками по Cout с заданным `block_size`.
- Input-channel sparsity: обнуляем входные каналы (Cin), уменьшается K = Cin*Kh*Kw.
Во всех режимах forward — FP16, backward — FP32, замеры только на CUDA.

**Метрики, которые считаются и выводятся:**
- `avg_forward_ms`, `avg_backward_ms`, `avg_step_ms` — среднее время (ms) на forward, backward и их сумму.
- `throughput_sps` — пропускная способность (семплов в секунду) по итоговому шагу.
- `speedup_forward`, `speedup_backward`, `speedup_step` (если выводятся) — отношение метрик Triton к torch; >1 — Triton быстрее.
- Ошибки вывода (Triton vs torch): `mae`, `max`, `rel_l2`.
- Память (когда собирается): `max_mem_alloc_mb`, `max_mem_reserved_mb`.
- Параметры разрежения: `mode` (`channel`, `block`, `input`), `keep_ratio` (доля оставленных каналов), `block_size` (для block sparsity).
- Конфигурация блоков Triton (если фиксируется): `BLOCK_M`, `BLOCK_N`, `BLOCK_K`, `NUM_WARPS`, `NUM_STAGES`.

**Как читать результаты:**
- Сравнивайте `avg_*` и `speedup_*`: >1 — быстрее torch, <1 — медленнее.
- Ошибки должны быть малыми (обычно 1e-4–1e-3 для fp16); рост ошибки при уменьшении `keep_ratio` сигнализирует о численной чувствительности.
- При уменьшении `keep_ratio` K или Cout сокращаются, но выгода зависит от выбранных BLOCK_* и паддинга K после img2col: иногда скорость растёт, иногда нет.


## Подготовка окружения
Ниже мы добавляем корень репозитория в `sys.path`, чтобы ноутбук, запущенный из папки `notebooks/`, мог импортировать пакет `conv_gemm`.

In [1]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path().resolve().parent))

## Импорты и настройки
В этой секции загружаем библиотеки и конфигурируем устройство/тип данных. По умолчанию вычисления идут в fp16 на GPU (если доступен CUDA), иначе падаем на CPU + fp32.

In [2]:
import time, copy, math
import torch
import pandas as pd
import torch.nn.functional as F
from torch.nn.utils import prune

from conv_gemm.baseline_layers.triton_conv2d import TritonConv2d as BaselineTritonConv2d

torch.backends.cudnn.benchmark = True
device = 'cuda' if torch.cuda.is_available() else 'cpu'
dtype = torch.float16 if device == 'cuda' else torch.float32
print(f'device: {device}, dtype: {dtype}')
print('Baseline Triton available:', BaselineTritonConv2d is not None)

device: cuda, dtype: torch.float16
Baseline Triton available: True


## Вспомогательные функции
- `sync_device` и `benchmark_module` — измерение времени (forward + backward).
- `compare_modules` — средняя/максимальная ошибка относительно PyTorch Conv2d.
- `finetune_module` — лёгкий тюнинг sparse модели (имитируем дистилляцию от dense-версии).

In [3]:
def sync_device():
    if device == 'cuda':
        torch.cuda.synchronize()

def clone_weights(dst, src):
    with torch.no_grad():
        dst.weight.copy_(src.weight)
        if dst.bias is not None and src.bias is not None:
            dst.bias.copy_(src.bias)

def compare_modules(ref, other, x):
    ref_out = ref(x).float()
    test_out = other(x).float()
    diff = (ref_out - test_out).abs()
    return {
        'mae': diff.mean().item(),
        'max': diff.max().item(),
        'rel_l2': diff.norm().item() / (ref_out.norm().item() + 1e-12)
    }

def benchmark_module(module, x, iters=50, warmup=10):
    module.eval()
    sync_device()
    with torch.no_grad():
        for _ in range(warmup):
            module(x)
    sync_device()
    start = time.perf_counter()
    with torch.no_grad():
        for _ in range(iters):
            module(x)
    sync_device()
    return (time.perf_counter() - start) * 1000.0 / iters

def finetune_module(module, teacher, steps=0, lr=1e-3, batch_shape=(16, 64, 56, 56), grad_clip=None):
    if steps <= 0:
        return
    orig_dtype = next(module.parameters()).dtype
    module.to(torch.float32)
    teacher_copy = copy.deepcopy(teacher).to(torch.float32).eval()
    module.train()
    opt = torch.optim.Adam(module.parameters(), lr=lr)
    for step in range(steps):
        x = torch.randn(*batch_shape, device=device, dtype=torch.float32)
        with torch.no_grad():
            target = teacher_copy(x)
        pred = module(x)
        loss = F.mse_loss(pred, target)
        opt.zero_grad(set_to_none=True)
        loss.backward()
        if grad_clip is not None:
            torch.nn.utils.clip_grad_norm_(module.parameters(), grad_clip)
        opt.step()
        if step % max(1, steps // 5) == 0:
            print(f'[finetune step {step}] loss={loss.item():.4e}')
    module.to(orig_dtype).eval()
    sync_device()


## Базовая конфигурация слоёв
Задаём параметры свёртки и создаём три реализации: PyTorch Conv2d, TritonConv2d (если доступен GPU).

In [4]:
params = dict(in_channels=1, out_channels=3, kernel_size=11, stride=1, padding=1, bias=True)
B, H, W = 16, 1024, 1024

torch_conv = torch.nn.Conv2d(**params).to(device=device, dtype=dtype)

baseline_block_cfg = dict(BLOCK_M=64, BLOCK_N=64, BLOCK_K=32, NUM_WARPS=4, NUM_STAGES=2)

def build_baseline(block_cfg=None):
    if device != 'cuda':
        return None
    cfg = block_cfg or baseline_block_cfg
    tri = BaselineTritonConv2d(
        **params,
        BLOCK_M=cfg['BLOCK_M'], BLOCK_N=cfg['BLOCK_N'], BLOCK_K=cfg['BLOCK_K'],
        NUM_WARPS=cfg['NUM_WARPS'], NUM_STAGES=cfg['NUM_STAGES']
    ).to(device)
    clone_weights(tri, torch_conv)
    return tri

tri_dense = build_baseline()

### Базовое сравнение без разрежения

Ниже таблица с метриками для `nn.Conv2d` и плотной `TritonConv2d` на фиксированных shape. Столбцы:
- `avg_forward_ms`, `avg_backward_ms`, `avg_step_ms`, `throughput_sps`;
- при наличии — `speedup_*` (torch / triton).
Эти значения служат опорной точкой для последующих экспериментов с разрежением.


In [5]:
x_sample = torch.randn(B, params['in_channels'], H, W, device=device, dtype=dtype)
records = []
modules = [('Torch Conv2d', torch_conv)]
if tri_dense is not None:
    modules.append(('Baseline Triton fp16', tri_dense))

for name, module in modules:
    stats = compare_modules(torch_conv, module, x_sample)
    t_ms = benchmark_module(module, x_sample.clone().detach())
    records.append({'layer': name, 'mae': stats['mae'], 'max': stats['max'], 'rel_l2': stats['rel_l2'], 'time_ms': t_ms})
baseline_df = pd.DataFrame(records)
baseline_df

,layer,mae,max,rel_l2,time_ms
0,Torch Conv2d,0.000000,0.000000,0.000000,11.749944
1,Baseline Triton fp16,0.000095,0.001953,0.000395,36.923735


### Channel sparsity: что измеряем

В этом прогоне маска применена к выходным каналам (Cout). Эффективное K не меняется, но уменьшается число столбцов в GEMM. Поля в таблице:
- `mode="channel"`, `keep_ratio` — доля оставленных каналов.
- Время: `avg_forward_ms`, `avg_backward_ms`, `avg_step_ms` (ms) и при наличии `speedup_*`.
- Ошибки: `mae`, `max`, `rel_l2`.
Ожидание: время должно снижаться с уменьшением `keep_ratio`, но при слишком малых значениях возможны численные шумы или снижение эффективности из‑за паддинга/тайлинга.


In [6]:
def run_channel_sweep(keep_ratios, finetune_steps=0, finetune_lr=1e-3, block_cfg=None, grad_clip=None, batch_shape=None):
    if tri_dense is None:
        raise RuntimeError('Triton недоступен (нужен GPU)')
    cfg = block_cfg or baseline_block_cfg
    batch_shape = batch_shape or (B, params['in_channels'], H, W)
    rows = []
    teacher = build_baseline(cfg) if finetune_steps > 0 else tri_dense
    for ratio in keep_ratios:
        tri = build_baseline(cfg)
        tri.set_channel_sparsity(ratio)
        if finetune_steps > 0 and teacher is not None:
            finetune_module(tri, teacher, steps=finetune_steps, lr=finetune_lr, batch_shape=batch_shape, grad_clip=grad_clip)
        x = torch.randn(*batch_shape, device=device, dtype=dtype)
        stats = compare_modules(torch_conv, tri, x)
        t_ms = benchmark_module(tri, x.clone().detach())
        rows.append({'mode': 'channel', 'keep_ratio': ratio, 'mae': stats['mae'], 'max': stats['max'],
                     'rel_l2': stats['rel_l2'], 'time_ms': t_ms,
                     'BLOCK_M': cfg['BLOCK_M'], 'BLOCK_N': cfg['BLOCK_N'],
                     'BLOCK_K': cfg['BLOCK_K'], 'NUM_WARPS': cfg['NUM_WARPS'], 'NUM_STAGES': cfg['NUM_STAGES']})
    return pd.DataFrame(rows)

def run_block_sweep(keep_ratios, block_size=4, block_cfg=None):
    if tri_dense is None:
        raise RuntimeError('Triton недоступен (нужен GPU)')
    cfg = block_cfg or baseline_block_cfg
    rows = []
    for ratio in keep_ratios:
        tri = build_baseline(cfg)
        tri.set_block_sparsity(ratio, block_size=block_size)
        x = torch.randn(B, params['in_channels'], H, W, device=device, dtype=dtype)
        stats = compare_modules(torch_conv, tri, x)
        t_ms = benchmark_module(tri, x.clone().detach())
        rows.append({'mode': f'block-{block_size}', 'keep_ratio': ratio, 'mae': stats['mae'], 'max': stats['max'],
                     'rel_l2': stats['rel_l2'], 'time_ms': t_ms,
                     'BLOCK_M': cfg['BLOCK_M'], 'BLOCK_N': cfg['BLOCK_N'],
                     'BLOCK_K': cfg['BLOCK_K'], 'NUM_WARPS': cfg['NUM_WARPS'], 'NUM_STAGES': cfg['NUM_STAGES']})
    return pd.DataFrame(rows)

def run_input_sweep(keep_ratios, block_cfg=None):
    if tri_dense is None:
        raise RuntimeError('Triton недоступен (нужен GPU)')
    cfg = block_cfg or baseline_block_cfg
    rows = []
    for ratio in keep_ratios:
        tri = build_baseline(cfg)
        tri.set_input_channel_sparsity(ratio)
        x = torch.randn(B, params['in_channels'], H, W, device=device, dtype=dtype)
        stats = compare_modules(torch_conv, tri, x)
        t_ms = benchmark_module(tri, x.clone().detach())
        rows.append({'mode': 'input', 'keep_ratio': ratio, 'mae': stats['mae'], 'max': stats['max'],
                     'rel_l2': stats['rel_l2'], 'time_ms': t_ms,
                     'BLOCK_M': cfg['BLOCK_M'], 'BLOCK_N': cfg['BLOCK_N'],
                     'BLOCK_K': cfg['BLOCK_K'], 'NUM_WARPS': cfg['NUM_WARPS'], 'NUM_STAGES': cfg['NUM_STAGES']})
    return pd.DataFrame(rows)

keep_ratios = [1.0, 0.85, 0.75, 0.65, 0.5, 0.35, 0.25]
channel_sweep_df = run_channel_sweep(keep_ratios)
channel_sweep_df

,mode,keep_ratio,mae,max,rel_l2,time_ms,BLOCK_M,BLOCK_N,BLOCK_K,NUM_WARPS,NUM_STAGES
0,channel,1.00,0.000095,0.001953,0.000395,38.914281,64,64,32,4,2
1,channel,0.85,0.000095,0.001953,0.000396,36.897463,64,64,32,4,2
2,channel,0.75,0.154193,3.255859,0.584438,37.233178,64,64,32,4,2
3,channel,0.65,0.154200,3.066406,0.584389,40.296615,64,64,32,4,2
4,channel,0.50,0.154207,3.197266,0.584336,35.775543,64,64,32,4,2
5,channel,0.35,0.303686,3.095703,0.814204,29.802220,64,64,32,4,2
6,channel,0.25,0.303770,3.076172,0.814110,32.495487,64,64,32,4,2


### Визуализация trade-off
Ниже строим таблицу с дополнительными метриками и вычисляем относительное ускорение по сравнению с плотным Triton.

In [7]:
has_dense = 'Baseline Triton fp16' in baseline_df['layer'].values
if has_dense:
    dense_time = baseline_df.loc[baseline_df['layer'] == 'Baseline Triton fp16', 'time_ms'].iloc[0]
else:
    dense_time = None

viz_df = channel_sweep_df.copy()
if dense_time is not None:
    viz_df['speedup_vs_dense'] = dense_time / viz_df['time_ms']
viz_df.sort_values('keep_ratio')

,mode,keep_ratio,mae,max,rel_l2,time_ms,BLOCK_M,BLOCK_N,BLOCK_K,NUM_WARPS,NUM_STAGES,speedup_vs_dense
6,channel,0.25,0.303770,3.076172,0.814110,32.495487,64,64,32,4,2,1.136273
5,channel,0.35,0.303686,3.095703,0.814204,29.802220,64,64,32,4,2,1.238959
4,channel,0.50,0.154207,3.197266,0.584336,35.775543,64,64,32,4,2,1.032094
3,channel,0.65,0.154200,3.066406,0.584389,40.296615,64,64,32,4,2,0.916299
2,channel,0.75,0.154193,3.255859,0.584438,37.233178,64,64,32,4,2,0.991689
1,channel,0.85,0.000095,0.001953,0.000396,36.897463,64,64,32,4,2,1.000712
0,channel,1.00,0.000095,0.001953,0.000395,38.914281,64,64,32,4,2,0.948848


### Block sparsity: что измеряем

Здесь обнуляем фильтры группами по `block_size` (обычно 4). Меняется эффективное число столбцов в GEMM группами. Поля:
- `mode="block"`, `keep_ratio`, `block_size`.
- Время и скорость (`avg_*`, `speedup_*`), ошибки (`mae`, `max`, `rel_l2`).
Смотрите, как выбор `block_size` и `keep_ratio` влияет на время: группировка может лучше совпадать с BLOCK_K и давать прирост, либо наоборот добавлять паддинг.


In [8]:
block_sweep_df = run_block_sweep(keep_ratios, block_size=4)
block_sweep_df

,mode,keep_ratio,mae,max,rel_l2,time_ms,BLOCK_M,BLOCK_N,BLOCK_K,NUM_WARPS,NUM_STAGES
0,block-4,1.00,0.000095,0.001953,0.000395,42.284613,64,64,32,4,2
1,block-4,0.85,0.000095,0.001953,0.000395,39.503017,64,64,32,4,2
2,block-4,0.75,0.000095,0.001953,0.000395,41.171895,64,64,32,4,2
3,block-4,0.65,0.000095,0.001953,0.000395,39.621034,64,64,32,4,2
4,block-4,0.50,0.000095,0.001953,0.000395,36.771493,64,64,32,4,2
5,block-4,0.35,0.000095,0.001953,0.000395,37.004154,64,64,32,4,2
6,block-4,0.25,0.000095,0.001953,0.000395,36.802770,64,64,32,4,2


### Input-channel sparsity: что измеряем

Маска на входных каналах (Cin) уменьшает K = Cin*Kh*Kw для img2col/GEMM/col2img. Поля:
- `mode="input"`, `keep_ratio`.
- Время (`avg_*`, `speedup_*`), ошибки (`mae`, `max`, `rel_l2`).
Ожидание: сокращение K чаще даёт ускорение, но при сильном урезании может вызывать паддинг K и ухудшение тайлинга. Проверяйте, что ошибки остаются малыми.


In [9]:
input_sweep_df = run_input_sweep([1.0,0.9,0.8,0.7,0.6,0.5,0.4,0.3,0.25])
input_sweep_df

,mode,keep_ratio,mae,max,rel_l2,time_ms,BLOCK_M,BLOCK_N,BLOCK_K,NUM_WARPS,NUM_STAGES
0,input,1.00,0.000095,0.001953,0.000395,38.992164,64,64,32,4,2
1,input,0.90,0.000095,0.001953,0.000396,42.357784,64,64,32,4,2
2,input,0.80,0.000095,0.001953,0.000395,36.763744,64,64,32,4,2
3,input,0.70,0.000095,0.001953,0.000395,37.023314,64,64,32,4,2
4,input,0.60,0.000095,0.001953,0.000395,36.711776,64,64,32,4,2
5,input,0.50,0.000095,0.001953,0.000395,36.549327,64,64,32,4,2
6,input,0.40,0.000095,0.001953,0.000396,36.553659,64,64,32,4,2
7,input,0.30,0.000095,0.001953,0.000395,36.401114,64,64,32,4,2
8,input,0.25,0.000095,0.001953,0.000396,39.581935,64,64,32,4,2


## Summary
Ниже сводим сравнение baseline vs. sparsity режимов (channel/block/input).

In [10]:
summary_frames = []
summary_frames.append(channel_sweep_df.assign(mode='channel'))
summary_frames.append(block_sweep_df.assign(mode='block'))
sum_input = input_sweep_df.assign(mode='input')
summary_frames.append(sum_input)
summary_df = pd.concat(summary_frames, ignore_index=True)
if 'Baseline Triton fp16' in baseline_df['layer'].values:
    dense_time = baseline_df.loc[baseline_df['layer'] == 'Baseline Triton fp16', 'time_ms'].iloc[0]
    summary_df['speedup_vs_dense'] = dense_time / summary_df['time_ms']
summary_df[['mode','keep_ratio','mae','max','time_ms','speedup_vs_dense']]

,mode,keep_ratio,mae,max,time_ms,speedup_vs_dense
0,channel,1.00,0.000095,0.001953,38.914281,0.948848
1,channel,0.85,0.000095,0.001953,36.897463,1.000712
2,channel,0.75,0.154193,3.255859,37.233178,0.991689
3,channel,0.65,0.154200,3.066406,40.296615,0.916299
4,channel,0.50,0.154207,3.197266,35.775543,1.032094
5,channel,0.35,0.303686,3.095703,29.802220,1.238959
6,channel,0.25,0.303770,3.076172,32.495487,1.136273
7,block,1.00,0.000095,0.001953,42.284613,0.873219
8,block,0.85,0.000095,0.001953,39.503017,0.934707
9,block,0.75,0.000095,0.001953,41.171895,0.896819


### Сводное сравнение режимов sparsity

Здесь собраны лучшие результаты по каждому режиму (`channel`, `block`, `input`) и значению `keep_ratio`. Смотрите на:
- `speedup_*` против torch (или против плотного Triton, если так выбрано);
- Ошибки (`mae`, `rel_l2`) — должны оставаться в допустимых пределах;
- Память (`max_mem_*`), если измерялась: уменьшение каналов часто снижает выделенную/резервируемую память.
Эти строки помогают выбрать режим и `keep_ratio`, дающие выгодный баланс точности и скорости.


In [11]:
summary_frames = []
summary_frames.append(channel_sweep_df.assign(mode='channel'))
summary_frames.append(block_sweep_df.assign(mode='block'))
summary_frames.append(input_sweep_df.assign(mode='input'))
summary_df = pd.concat(summary_frames, ignore_index=True)
if 'Baseline Triton fp16' in baseline_df['layer'].values:
    dense_time = baseline_df.loc[baseline_df['layer']=='Baseline Triton fp16','time_ms'].iloc[0]
    summary_df['speedup_vs_dense'] = dense_time / summary_df['time_ms']
else:
    summary_df['speedup_vs_dense'] = float('nan')
summary_df[['mode','keep_ratio','mae','max','time_ms','speedup_vs_dense']]

,mode,keep_ratio,mae,max,time_ms,speedup_vs_dense
0,channel,1.00,0.000095,0.001953,38.914281,0.948848
1,channel,0.85,0.000095,0.001953,36.897463,1.000712
2,channel,0.75,0.154193,3.255859,37.233178,0.991689
3,channel,0.65,0.154200,3.066406,40.296615,0.916299
4,channel,0.50,0.154207,3.197266,35.775543,1.032094
5,channel,0.35,0.303686,3.095703,29.802220,1.238959
6,channel,0.25,0.303770,3.076172,32.495487,1.136273
7,block,1.00,0.000095,0.001953,42.284613,0.873219
8,block,0.85,0.000095,0.001953,39.503017,0.934707
9,block,0.75,0.000095,0.001953,41.171895,0.896819


### Top-10 configurations by speedup

In [12]:
top10 = summary_df.dropna(subset=['speedup_vs_dense']).sort_values('speedup_vs_dense', ascending=False).head(10)
top10[['mode','keep_ratio','time_ms','speedup_vs_dense','mae','max']]

,mode,keep_ratio,time_ms,speedup_vs_dense,mae,max
5,channel,0.35,29.802220,1.238959,0.303686,3.095703
6,channel,0.25,32.495487,1.136273,0.303770,3.076172
4,channel,0.50,35.775543,1.032094,0.154207,3.197266
21,input,0.30,36.401114,1.014357,0.000095,0.001953
19,input,0.50,36.549327,1.010244,0.000095,0.001953
20,input,0.40,36.553659,1.010124,0.000095,0.001953
18,input,0.60,36.711776,1.005774,0.000095,0.001953
16,input,0.80,36.763744,1.004352,0.000095,0.001953
11,block,0.50,36.771493,1.004140,0.000095,0.001953
13,block,0.25,36.802770,1.003287,0.000095,0.001953
